In [ ]:
import tensorflow as tf
import numpy as np
import datetime
import os
import glob

In [ ]:
MODEL_CP_DIR = '/content/drive/My Drive/SE+/DenseNet/model-{epoch:03d}-{loss:03f}-{val_loss:03f}.h5'
MODEL_LOG_DIR ='/content/drive/My Drive/SE+/DenseNet/logs'
MODEL_AUC_LOG_DIR = '/content/drive/My Drive/SE+/DenseNet/logs/aucperepoch.csv'

MODEL_ID=''

INIT_LR = 1e-4
BS=32
WK=5
EPOCH=20
SELECTED_VIEW = 'pa_ll'

AUC_BS = 512

## **Data Pipeline**

### Image Augmentation

In [ ]:
import math
from tensorflow.keras import backend as K

def get_mat(rotation,height_zoom=1,width_zoom=1):
    rotation = math.pi * rotation / 180.
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    return K.dot(rotation_matrix, zoom_matrix)

In [ ]:
def transform(image):
    DIM = 224
    XDIM = DIM%2  
    rot = 10. * tf.random.normal([1],dtype='float32')
    m = get_mat(rot) 
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
    return tf.reshape(d,[DIM,DIM,3])

In [ ]:
def augment_img(image):
  image = tf.image.resize(image, [264,264])
  image = tf.image.random_brightness(image, 0.4)
  image = tf.image.random_crop(image, [224,224,3])
  image = tf.image.random_flip_left_right(image)
  image = transform(image)
  return image

### TFrecord

In [ ]:
def augment_img_wot(image):
  image = tf.image.resize(image, [264,264])
  image = tf.image.crop_to_bounding_box(image, 20,20, 224,224)
  return image

In [ ]:
def read_tfrecord(example, output_mode='both', with_transform=False):
    features = {
        "image_front": tf.io.FixedLenFeature([], tf.string),
        "image_side": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.VarLenFeature(tf.float32),  
        "view": tf.io.FixedLenFeature([], tf.string),
        "select": tf.io.FixedLenFeature([], tf.int64),
        "study_id": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, features)
    label  = tf.reshape(tf.sparse.to_dense(example['label']), [1, 14])
    label = tf.where(tf.math.is_nan(label), tf.ones_like(label) * 0, label)
    label = tf.where(label == -1, tf.ones_like(label) * 0, label)
    label = label[0]

    select = example['select']
    if output_mode=='label':
      return label, select
    elif output_mode=='view_label':
      return label, example['view'], select
    elif output_mode=='combine_label':
      return (label,label, label), select
    image_front = tf.image.decode_jpeg(example['image_front'], channels=3)
    image_side =  tf.image.decode_jpeg(example['image_side'], channels=3)

    if with_transform:
      image_front = augment_img(image_front)
      image_side = augment_img(image_side)
    else: 
      image_front = augment_img_wot(image_front)
      image_side = augment_img_wot(image_side)
    if output_mode=='both':
      return [image_front, image_side], label, select
    elif output_mode=='img_view':
      return [image_front, image_side], example['view'], select
    elif output_mode=='both_view':
      return [image_front, image_side], label, example['view'], select
    else:
      return [image_front, image_side], select

In [ ]:
option_no_order = tf.data.Options()
option_no_order.experimental_deterministic = False

### Image Selection and Preprocessing

In [ ]:
def select_image(*record):
  select = record[-1]
  return select == 1

def remove_select(*record):
  return record[:-1]

In [ ]:
def select_view_cat(view, *record):
  current_view = record[-1]
  return current_view == view

def remove_view(*record):
  return record[:-1]

In [ ]:
def select_view(view, *record):
  record = record[0]
  img = None
  if view == 'frontal':
    img = record[0][0]
  elif view == 'side':
    img = record[0][1]
  else: 
    img = record[0]
  if len(record) == 1:
    return img
  return img, record[1]

In [ ]:
def preprocess_image(image):
  image = tf.cast(image, tf.float32)
  image = image / 255.
  image_net_mean = np.array([0.485, 0.456, 0.406])
  image_net_std = np.array([0.229, 0.224, 0.225])
  image = tf.math.subtract(image, image_net_mean)
  image = tf.math.divide(image, image_net_std)
  return image
def preprocess_image_dataset(image, label=None, replicate_label=False):
  image = preprocess_image(image)
  if len(image.shape) ==5:
    image = (image[:,0], image[:,1])
  if label is None:
    return image
  if replicate_label:
    label = (label, label, label)
  return image, label

In [ ]:
def choose_view(ds, view):
  ds = ds.filter(lambda *record: select_view_cat(view, *record)).map(remove_view, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  return ds

In [ ]:
def choose_selected(ds):
  ds = ds.filter(select_image).map(remove_select, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  return ds

### Data loader

In [ ]:
filenames = tf.io.gfile.glob('/content/drive/My Drive/AG-CNN/tfrecord_train_mul_nr/*.tfrec')
train_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
train_dsrl = train_dsr.shuffle(300000).with_options(option_no_order)
train_dsrl = train_dsrl.map(lambda record : read_tfrecord(record, 'both', True), num_parallel_calls=tf.data.experimental.AUTOTUNE)
#  select only images that were randomly selected
train_dsrs = choose_selected(train_dsrl)

In [ ]:
filenames = tf.io.gfile.glob('/content/drive/My Drive/AG-CNN/tfrecord_val_mul_nr/*.tfrec')
val_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
val_dsrl = val_dsr.map(read_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)

#  select only images that were randomly selected
val_dsrs = choose_selected(val_dsrl)

In [ ]:
train_dsrl = train_dsr.shuffle(300000).with_options(option_no_order)

train_dsrs_view = train_dsrl.map(lambda record : read_tfrecord(record, 'both_view'), num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dsrs_view = val_dsr.map(lambda record : read_tfrecord(record, 'both_view'), num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dsrs_view = choose_selected(train_dsrs_view)
val_dsrs_view = choose_selected(val_dsrs_view)

train_dsrs_view = choose_view(train_dsrs_view, SELECTED_VIEW).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
val_dsrs_view = choose_view(val_dsrs_view, SELECTED_VIEW).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
filenames = tf.io.gfile.glob('/content/drive/My Drive/AG-CNN/tfrecord_test_mul_nr/*.tfrec')
test_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)
test_dsrl = test_dsr.map(read_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)

test_dsrs_img = test_dsr.map(lambda record : read_tfrecord(record, 'img'), num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dsrs_lab = test_dsr.map(lambda record : read_tfrecord(record, 'label'), num_parallel_calls=tf.data.experimental.AUTOTUNE)

#  select only images that were randomly selected
test_dsrs = choose_selected(test_dsrl)
test_dsrs_img = choose_selected(test_dsrs_img)
test_dsrs_lab = choose_selected(test_dsrs_lab)

In [ ]:
filenames = tf.io.gfile.glob('/content/drive/My Drive/AG-CNN/tfrecord_test_mul_nr/*.tfrec')
test_dsr = tf.data.TFRecordDataset(filenames, num_parallel_reads=tf.data.experimental.AUTOTUNE)

test_dsrs_img_view = test_dsr.map(lambda record : read_tfrecord(record, 'both_view'), num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dsrs_lab_view = test_dsr.map(lambda record : read_tfrecord(record, 'view_label'), num_parallel_calls=tf.data.experimental.AUTOTUNE)

test_dsrs_img_view = choose_selected(test_dsrs_img_view)
test_dsrs_lab_view = choose_selected(test_dsrs_lab_view)

test_dsrs_img_view = choose_view(test_dsrs_img_view, SELECTED_VIEW).batch(AUC_BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
test_dsrs_lab_view = choose_view(test_dsrs_lab_view, SELECTED_VIEW).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
combine_train_ds = train_dsrs.map(lambda *record : select_view('both', record)).batch(BS)

# two side images, one label
combine_train_dsr = combine_train_ds.map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
frontal_train_dsr = train_dsrs.map(lambda *record : select_view('frontal',record)).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
side_train_dsr = train_dsrs.map(lambda *record : select_view( 'side',record)).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
combine_val_ds = val_dsrs.map(lambda *record : select_view('both', record)).batch(BS)

combine_val_dsr = combine_val_ds.map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
frontal_val_dsr = val_dsrs.map(lambda *record : select_view('frontal',record)).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
side_val_dsr = val_dsrs.map(lambda *record : select_view( 'side', record)).batch(BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
combine_test_dsr_img = test_dsrs.map(lambda *record : select_view( 'both', record)).batch(AUC_BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
frontal_test_dsr_img = test_dsrs.map(lambda *record : select_view( 'frontal',  record)).batch(AUC_BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
side_test_dsr_img = test_dsrs.map(lambda *record : select_view('side', record)).batch(AUC_BS).map(preprocess_image_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)

test_dsr_lab = test_dsrs_lab.prefetch(tf.data.experimental.AUTOTUNE)
# labels for specific view
frontal_test_dsr_lab = test_dsrs.map(lambda *record : select_view( 'frontal', record)).map(lambda img, lab: lab, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
side_test_dsr_lab  = test_dsrs.map(lambda *record : select_view('side',record)).map(lambda img, lab: lab, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)

## **Model creation**

### Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, GlobalAveragePooling2D,GlobalMaxPooling2D, concatenate, Lambda, Multiply
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121

def load_base_model(input, name):
  base_model = DenseNet121(weights="imagenet", include_top=False,
	input_tensor=input)
  for layer in base_model.layers:
    layer._name = layer.name + str(name)
  return base_model

In [ ]:
def se_block(x, ratio=16):
  channels = x.shape[-1]

  head = GlobalAveragePooling2D()(x)
  head = Dense(channels // ratio, activation='relu')(head)
  head = Dense(channels, activation='sigmoid')(head)

  head = Multiply()([x, head])

  return head

In [ ]:
def create_main_model():
  input_front = Input(shape=(224,224,3), name = 'front_input')
  input_side = Input(shape=(224,224,3), name = 'side_input')

  base_model_front = load_base_model(input_front, "_front")
  base_model_side = load_base_model(input_side, "_side")

  headModel1 = base_model_front.output
  headModel2 = base_model_side.output

  channels = headModel1.shape[-1]

  combined_output = concatenate([headModel1, headModel2], axis=-1)
  combined_output = se_block(combined_output)

  headModel1 = Lambda(lambda x: x[:,:,:,:channels])(combined_output)
  headModel2 = Lambda(lambda x: x[:,:,:,channels:])(combined_output)

  headModel1 = GlobalMaxPooling2D(name='front_conv')(headModel1)
  headModel2 = GlobalMaxPooling2D(name='side_conv')(headModel2)
  headModelCombine = concatenate([headModel1, headModel2], axis=-1)
  headModelCombine = Dense(14, activation="sigmoid", name='combined_output')(headModelCombine)

  output = headModelCombine
  input = [input_front, input_side]

  model = Model(inputs=input, outputs=output)

  optimizer = Adam(learning_rate=INIT_LR)
  model.compile(loss="binary_crossentropy", optimizer=optimizer,
	  metrics=[tf.keras.metrics.AUC(multi_label=True)])
  return model

In [ ]:
main_model = create_main_model()

29089792/29084464 [==============================] - 0s 0us/step


In [ ]:
# main_model.summary()

### Callbacks

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping, ModelCheckpoint, TensorBoard

In [ ]:
from sklearn.metrics import roc_auc_score
import csv

class AUCCallback(tf.keras.callbacks.Callback):
    def __init__(self, log_dir, img, lab):
      self.img_set = img
      self.lab_set = lab
      self.log_dir = log_dir
        
    def on_epoch_end(self, epoch, logs={}):
      pred = self.model.predict(self.img_set, verbose=1)
      rocs = np.zeros(14)
      for j in range(14):
        rocs[j] = roc_auc_score(self.lab_set[:,j], pred[:,j])
        
      with open(self.log_dir, 'a') as fp:
        writer = csv.writer(fp, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(rocs)
      return

In [ ]:
def create_auc_logger(logdir, auc_test_img, auc_test_lab):
  test_label = list(auc_test_lab)
  test_label = np.array(tf.stack(test_label))
  test_label = test_label[:,0,:]
  return AUCCallback(logdir, auc_test_img,test_label)

In [ ]:
def set_callback(tblogdir, checkpoint_address=None, auc_log_dir=None, auc_test_img=None, auc_test_lab=None):
  tensorboard_callback = TensorBoard(tblogdir, histogram_freq=1)
 
  callbacks = [
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=1e-6),
             tensorboard_callback
  ]

  if checkpoint_address:
    checkpoint_address += 'model-{epoch:03d}-{loss:03f}-{val_loss:03f}.h5'
    callbacks += [
            EarlyStopping(patience=3, verbose=1),
            ModelCheckpoint(checkpoint_address, save_best_only=True, monitor='val_loss')
    ]

  if auc_log_dir:
    auc_logger = create_auc_logger(auc_log_dir,auc_test_img,auc_test_lab)
    callbacks.append(auc_logger)
  return callbacks

## **Training Process**

### Baseline Training

In [ ]:
tblogdir = os.path.join(MODEL_LOG_DIR, datetime.datetime.now().strftime("%Y%m%d") + '-model' + MODEL_ID)
tensorboard_callback = TensorBoard(tblogdir, histogram_freq=1)

In [ ]:
auc_callback = create_auc_logger(MODEL_AUC_LOG_DIR, combine_test_dsr_img, test_dsrs_lab)

In [ ]:
callbacks = [
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=1e-6),
             EarlyStopping(patience=3, verbose=1),
             ModelCheckpoint(MODEL_CP_DIR, save_best_only=True, monitor='val_loss'),
             tensorboard_callback,
             auc_callback
  ]

In [ ]:
history = main_model.fit(
        combine_train_dsr,
        epochs=EPOCH, 
        validation_data= combine_val_dsr,
        callbacks=callbacks,
        use_multiprocessing=True,
        workers=WK
    )

Epoch 1/20
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2298/2298 [==============================] - 1135s 494ms/step - loss: 0.2100 - auc: 0.7509 - val_loss: 0.1897 - val_auc: 0.8037 - lr: 1.0000e-04
Epoch 2/20
2298/2298 [==============================] - 1135s 494ms/step - loss: 0.1905 - auc: 0.7982 - val_loss: 0.1875 - val_auc: 0.8095 - lr: 1.0000e-04
Epoch 3/20
2298/2298 [==============================] - 1141s 496ms/step - loss: 0.1865 - auc: 0.8121 - val_loss: 0.1864 - val_auc: 0.8185 - lr: 1.0000e-04
Epoch 4/20
2298/2298 [==============================] - 1139s 496ms/step - loss: 0.1832 - auc: 0.8210 - val_loss: 0.1825 - val_auc: 0.8228 - lr: 1.0000e-04
Epoch 5/20
2298/2298 [==============================] - 1132s 493ms/step - loss: 0.1808 - auc: 0.8279 - val_loss: 0.1816 - val_auc: 0.8301 - lr: 1.0000e-04
Epoch 6/20
2298/2298 [==============================] - 1134s 493ms/step - loss: 0.1782 - auc: 0.8355 - val_loss: 0.1804 - val_auc: 0.8321 

In [ ]:
model = tf.keras.models.load_model("/content/drive/My Drive/SE/DenseNet/model2-007-0.167040-0.176531.h5")

In [ ]:
test_label = list(test_dsrs_lab_view)
test_label = np.array(tf.stack(test_label))
test_label = test_label[:,0,:]

In [ ]:
pred = model.predict(test_dsrs_img_view, verbose=1, workers=WK, use_multiprocessing=True)
rocs = np.zeros(14)
for j in range(14):
  rocs[j] = roc_auc_score(test_label[:,j], pred[:,j])

      4/Unknown - 13s 3s/step

In [ ]:
with open('/content/drive/My Drive/test.csv', 'a') as fp:
  writer = csv.writer(fp, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  writer.writerow(rocs)